In [1]:
import os

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

sns.set()


PyPop7: A Pure-Python Library for POPulation-based Black-Box Optimization (BBO)

Documentation
[https://pypop.readthedocs.io/en/latest/](https://pypop.readthedocs.io/en/latest/)

Repository
[https://github.com/Evolutionary-Intelligence/pypop](https://github.com/Evolutionary-Intelligence/pypop)

 Install using:
 `pip install pypop7`

In [2]:
# Success-History based Adaptive Differential Evolution (SHADE).
from pypop7.optimizers.de import SHADE

In [3]:
# project files
os.environ.setdefault("RP_MODEL_FILE_PATH", "../files")

from rp_model.utils import pack, unpack, table, load, digest
from rp_model.calc import FitOptions, compute_rp, make_precomputed_columns, make_initial_guess

Configured files path to: C:\Users\RaenonX\Documents\Projects\PokemonSleep\pokemon-sleep-rp\files


In [4]:
# stuff for display

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 120)

In [5]:
data = pd.read_pickle(FitOptions.data_file)
data.describe()
data.head()

,Level,RP,MS lvl,ModelRP,Difference,Freq1,FreqL,Inv,Berry1,BerryL,Ing1P,Helps per hour,Helps Neutral,NrgMult,IngrMult,SkillMult,Ingr%,SklContr,BerryD,IngD,Dupes,Amnt,Ing2P,Help skill bonus,RP Multiplier
count,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.0,8763.000000,8763.000000,8763.000000,8763.000000
mean,12.298756,688.924113,1.591464,688.891361,-0.032751,3831.016775,3661.778153,12.673057,28.063334,39.561109,108.767431,1.067699,0.987262,0.994203,0.998859,1.007600,0.208704,35.974795,58.788885,150.006048,1.0,0.142189,6.091749,0.984223,1.033434
std,8.124838,466.973756,1.069884,467.017787,0.633609,988.946537,1056.799659,3.621096,2.615509,9.214666,16.055961,0.327917,0.058253,0.044956,0.119072,0.119352,0.046418,40.872172,30.362783,64.954629,0.0,0.697996,29.183804,0.042143,0.075512
min,1.000000,245.000000,1.000000,245.000000,-17.000000,2200.000000,1489.118400,7.000000,24.000000,24.000000,90.000000,0.520000,0.890000,0.920000,0.800000,0.800000,0.130000,5.940000,25.000000,90.000000,1.0,0.000000,0.000000,0.790000,1.000000
25%,6.000000,371.000000,1.000000,371.000000,0.000000,3000.000000,2766.110800,10.000000,26.000000,33.000000,98.000000,0.820000,1.000000,1.000000,1.000000,1.000000,0.180000,14.050000,37.000000,101.000000,1.0,0.000000,0.000000,1.000000,1.000000
50%,10.000000,525.000000,1.000000,525.000000,0.000000,3800.000000,3626.000000,12.000000,28.000000,37.000000,103.000000,0.990000,1.000000,1.000000,1.000000,1.000000,0.210000,20.750000,50.000000,121.000000,1.0,0.000000,0.000000,1.000000,1.000000
75%,16.000000,811.000000,2.000000,811.000000,0.000000,4500.000000,4382.400000,15.000000,31.000000,44.000000,115.000000,1.300000,1.000000,1.000000,1.000000,1.000000,0.230000,47.760000,70.000000,202.000000,1.0,0.000000,0.000000,1.000000,1.000000
max,45.000000,3827.000000,6.000000,3830.000000,9.000000,6300.000000,6916.140000,24.000000,35.000000,92.000000,151.000000,2.410000,1.120000,1.080000,1.200000,1.200000,0.430000,414.490000,246.000000,511.000000,1.0,8.000000,342.000000,1.000000,1.440000


,Pokemon,Level,RP,Nature,MS lvl,Source,ModelRP,Difference,Nature2,ID,NatureP,NatureN,Freq1,FreqL,Type,Class,MSkill,Inv,Berry1,BerryL,Ing1,Ing1P,Helps per hour,Helps Neutral,NrgMult,IngrMult,SkillMult,Ingr%,SklContr,RPneutral,IDNeutral,BerryD,IngD,Dupes,DupeMatch,Sub Skill 1,Sub Skill 2,Ingredient 2,Amnt,Ing2P,Help skill bonus,RP Multiplier
0,Arbok,8.0,531.0,Hardy,1.0,Questions help guides,531.0,0.0,Neutral,ArbokNeutral81,-,-,3700.0,3648.20,Poison,Berries,Charge Energy S,14.0,32.0,39,Bean Sausage,103.0,0.98,1.0,1.00,1.0,1.0,0.26,22.75,531.00,ArbokNeutral81,78.0,103.0,1,True,,,,0.0,0.0,1.0,1.0
1,Arbok,8.0,538.0,Quiet,1.0,Questions help guides,538.0,0.0,Quiet,ArbokQuiet81,Ingredient Finding,Exp Gains,3700.0,3648.20,Poison,Berries,Charge Energy S,14.0,32.0,39,Bean Sausage,103.0,0.98,1.0,1.00,1.2,1.0,0.26,22.75,530.61,ArbokNeutral81,78.0,103.0,1,True,,,,0.0,0.0,1.0,1.0
2,Arbok,9.0,528.0,Bold,1.0,,528.0,0.0,Bold,ArbokBold91,Energy Recovery,Speed of Help,3700.0,4004.88,Poison,Berries,Charge Energy S,14.0,32.0,40,Bean Sausage,103.0,0.89,1.1,1.08,1.0,1.0,0.26,22.75,538.33,ArbokNeutral91,80.0,103.0,1,True,,,,0.0,0.0,1.0,1.0
3,Arbok,9.0,539.0,Hardy,1.0,Double Jumper,539.0,0.0,Neutral,ArbokNeutral91,-,-,3700.0,3640.80,Poison,Berries,Charge Energy S,14.0,32.0,40,Bean Sausage,103.0,0.98,1.0,1.00,1.0,1.0,0.26,22.75,539.00,ArbokNeutral91,80.0,103.0,1,True,,,,0.0,0.0,1.0,1.0
4,Arbok,9.0,574.0,Naughty,1.0,Rate My Mon,574.0,0.0,Naughty,ArbokNaughty91,Speed of Help,Main Skill Chance,3700.0,3276.72,Poison,Berries,Charge Energy S,14.0,32.0,40,Bean Sausage,103.0,1.09,0.9,1.00,1.0,0.8,0.26,22.75,538.35,ArbokNeutral91,80.0,103.0,1,True,,,,0.0,0.0,1.0,1.0


In [6]:
# Make an initial guess
# Pack the dictionary into a 1-D vector.
# Also store information on how to unpack that vector

x0, unpack_info = pack(*make_initial_guess())
computed = make_precomputed_columns(data)

KeyError: 'Sub Skill 3'

In [ ]:
# We compute the RP model in rp_model.py
# Here we create closure to glue the pieces together.

def RP(x):
    return compute_rp(x, data, computed, unpack_info)


# for the error analysis page,
# we'll compute the residual with exact rounding.

def residual(x):
    return data["RP"] - np.round(RP(x))


def sum_squares(x, **kwargs):
    r = residual(x)
    return np.dot(r, r)


In [ ]:
filename = FitOptions.get_result_file(digest(data, x0))
opt = load(filename)
opt

In [ ]:
inject_x0 = False,

# For this library, those are more a sampling region than real constraints
# Init between 0.5x and 1.5x. But swap bound if value is negative

optx = opt.x
choice_list = [0.5 * optx, 1.5 * optx]
bounds_low = np.select([optx >= 0, optx < 0], choice_list)
bounds_high = np.select([optx < 0, optx >= 0], choice_list)

problem = dict(fitness_function=sum_squares, ndim_problem=len(optx), upper_boundary=bounds_high,
    lower_boundary=bounds_low)
options = dict(max_function_evaluations=10000)

if not inject_x0:
    algo = SHADE(problem, options)

else:

    class ShadeCustom(SHADE):
        def initialize(self, args=None):
            x, y, a = SHADE.initialize(self, args)

            if (args is not None and 'x0' in args):
                x[0] = args['x0']
                y[0] = self._evaluate_fitness(x[0], args)

            return (x, y, a)


    algo = ShadeCustom(problem, options)

result_de = algo.optimize(fitness_function=None, args={"x0": optx})

In [ ]:
table(result_de)

In [ ]:
r_opt = data["RP"] - np.round(RP(result_de['best_so_far_x']))
pd.Series(r_opt).describe()

_ = plt.figure()
g = sns.histplot(x=np.clip(np.abs(r_opt), 1e-1, 1e3), log_scale=True, bins=20)
g.axes.set_yscale('log')
_ = g.bar_label(g.containers[1], labels=[str(v) if v else '' for v in g.containers[1].datavalues])

_ = plt.figure()
g = sns.histplot(x=r_opt)
_ = g.axes.set_yscale('log')
_ = g.bar_label(g.containers[1], labels=[str(v) if v else '' for v in g.containers[1].datavalues])

In [ ]:
sol = unpack(opt.x, unpack_info)
table(sol)